In [1]:
import pandas as pd
import requests
import os
import time

matches = pd.read_csv('WCMatches.csv')
matches.columns
matches[["winnerid","loserid"]]




,winnerid,loserid
0,7d05pfwn,jfvflnna
1,o8q1tnkp,o8q1tnkp
2,akq5t76h,akq5t76h
3,ng9mpb4s,ng9mpb4s
4,ng9mpb4s,ng9mpb4s
...,...,...
2783,dq44xncy,222vn7x7
2784,g7umra42,g7umra42
2785,g7umra42,222vn7x7
2786,g7umra42,dq44xncy


In [2]:
# Ensure unique_ids is unique
unique_ids = pd.concat([matches['winnerid'], matches['loserid']]).unique()
print(unique_ids.size)
player_info = pd.DataFrame()

1210


In [11]:
import os
# Load existing player_info if it exists, otherwise create empty DataFrame
player_info_path = "EloPlayerinfo.csv"
if os.path.exists(player_info_path):
    player_info = pd.read_csv(player_info_path, low_memory=False)
    processed_ids = set(player_info['player_id'].unique())
else:
    player_info = pd.DataFrame()
    processed_ids = set()

# Only process player_ids that are not already in player_info
to_process = [pid for pid in unique_ids if pid not in processed_ids]

for idx, player_id in enumerate(to_process, start=len(processed_ids)):
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #add player id
    merged_df.insert(loc=0, column= "player_id", value=player_id)

    # Append merged_df to player_info
    player_info = pd.concat([player_info, merged_df], ignore_index=True)

    print(f"Index: {idx}, Player ID: {player_id}")
    time.sleep(0.5) #wait 0.5 second to not pummel the server

# Save player_info after all data is collected
player_info.to_csv("EloPlayerinfo.csv", index=False)
print("List done")

List done


In [12]:
#find if new id's that have not been in the list before have come up
unique_ids_update = pd.concat([player_info['player_id'], player_info['opp_id']]).unique()
print(unique_ids_update.size)

90506
